# Dynamic Programming


## Fibonacci Number

Fibonacci number uses recursion to solve the subproblem first and then get to its original problem. Strictly speaking, this is not dynamic programming because no maximum/minimum problem is involved. But the idea is very similar.


\begin{align}
f(n+2) &= f(n+1) + f(n) \\
f(1) &= 1 \\
f(2) &= 1
\end{align}


Naive recursion is not efficient as its time complexity is $O(n^2)$.

In [6]:
def fibonacci(n):
    if n<=2:
        return 1

    return fibonacci(n-1) + fibonacci(n-2)

# test
print(fibonacci(1))
print(fibonacci(2))
# this will take about 18 seconds
print(fibonacci(40))


1
1
102334155


When we draw the recursion tree, it is easy to find out the reason, which is that there are duplicated subproblems.

For example, when calculating `f(20)`, we have to calculate `f(18)` and `f(19)`. To calculate `f(19)`, we have to calculate `f(18)` and `f(17)`. In this way, the subproblem `f(18)` has been calculated twice.
The time complexity of the naive implementation is $O(n^2)$, because there are $n$ subproblems, and each subproblem requires $O(n)$ time.

For **`duplicated subproblems`**, we can create a memo, which reduces the complexity to $O(n)$
- once the subproblem is done, we save its results to the memo 
- once the subproblem is needed, we go to the memo first before solving it directly.


In [9]:
def fibonacci(n):
    
    # initialize a memo
    memo = {}
    memo[1] = 1
    memo[2] = 1

    # define a recursive helper function
    def helper(n):
        # if n is not in memo, then calculate it
        if n not in memo:
            memo[n] = helper(n-1) + helper(n-2)
        # return the value of n from memo
        return memo[n]
    
    return helper(n)

# test
print(fibonacci(1))
print(fibonacci(2))
# this now is super fast
print(fibonacci(40))   


1
1
102334155


Another method is using **`dynamic programming (DP) table`**. The idea is same as the memo.

**`DP Table`** calculates `f(n)` from bottom to top, or from the smallest subproblem to the originam subproblem.
For example, to calculate `f(20)`, `DP` calculates `f(1), f(2), ..., f(20)` in sequence.

In [10]:
def fibonacci(n):
    # intialize a dp table
    dp = {i: 0 for i in range(1, n+1)}
    dp[1] = 1
    dp[2] = 1

    # solve subproblem from small to large
    for i in range(3, n+1):
        dp[i] = dp[i-1] + dp[i-2]
    
    return dp[n]

# test
print(fibonacci(1))
print(fibonacci(2))
# this now is super fast
print(fibonacci(40))

1
1
102334155


we can keep optimize the above method as only two values in the table are used at a time. There is no need to store all of them.


In [11]:
def fibonacci(n):
    dp_1 = 1
    dp_2 = 1

    for i in range(3, n+1):
        dp_1, dp_2 = dp_1 + dp_2, dp_1
    
    return dp_1

# test
print(fibonacci(1))
print(fibonacci(2))
# this now is super fast
print(fibonacci(40))

1
1
102334155


## Coin Change

You are given an integer array `coins` representing coins of different denominations and an integer `amount` representing a total amount of money.

Return the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return `-1`.

You may assume that you have an infinite number of each kind of coin.

Example 1:
```
Input: coins = [1,2,5], amount = 11
Output: 3
Explanation: 11 = 5 + 5 + 1
```

This could be treated as a DP problem, as each subproblem is independent.

### Naive Recursion
We can construct a recusion by recursively reducing the amount by the available coin in `coins`, which leads to a new `amount` as a subproblem. 

For a DP problem, we need:
- **base case**: `amount` is 0, and if it's negative then the amount cannot be made up by any combinations of the `coins`.
- **state, what will be changed in the origninal problem and its subproblem**: in this case, it is the `amount`
- **choice, what are choices that lead to the state changes**: the selection of coins 
- **DP function/table definition**:
  - from `top-to-down` recursion, we will use a `DP function`, which typically takes the state as an argument, and return what is required, such as the number of coins.
  - from `bottom-to-up`, we will use a `DP table`, which serves as a memo to avoid duplicated calculations.


The state change function can be:

\begin{align}
  f(x) = \begin{cases}
              1 && x = 0 \\
              -1 && x \lt 0 \\
              \min \{f(x - c) + 1| c \in coins\} && x > 0 
              \end{cases}

\end{align}

In [13]:
def coinChange(coins, amount):

    # dp function: get the minimum number of coins to make up amount
    def dp(amount):
        # if amount is 0, then return 0
        if amount == 0:
            return 0
        # if amount is negative, then return -1
        if amount < 0:
            return -1
        
        min_coins = float('inf')
        for coin in coins:
            # subproblem: get the minimum number of coins to make up amount - coin
            subproblem = dp(amount - coin)
            # if subproblem is unsolvable (-1), then skip it
            if subproblem == -1:
                continue
            # if subproblem is not -1, then update min_coins
            min_coins = min(min_coins, subproblem + 1) 
        
        return min_coins
    
    res = dp(amount)
    return res if res != float('inf') else -1

# test
# 3
print(coinChange([1,2,5], 11))
# -1
print(coinChange([2], 3))
# 0
print(coinChange([1], 0))

3
-1
0


### Memoization

Apparently, the above recursion has duplicated subproblems. We can use a memo to avoid duplicated calculations.


In [22]:
def coinChange(coins, amount):
    # initialize a memo to store the minimum number of coins to make up amount
    memo = {}

    # dp function: get the minimum number of coins to make up amount
    def dp(amount):
        # base cases
        if amount == 0:
            return 0
        if amount < 0:
            return -1

        # 
        min_amount = float('inf')
        # solve subproblems
        for coin in coins:
            # if amount - coin is not in memo, then calculate it and save it in memo
            if amount - coin not in memo:
                subproblem = dp(amount - coin)
                memo[amount-coin] = subproblem
            # if subproblem is unsolvable (-1), then skip it
            if memo[amount-coin] == -1:
                continue

            # update min_amount - state transition
            min_amount = min(min_amount, memo[amount-coin] + 1)

        # return the min_coins that can make up amount   
        memo[amount] = min_amount

        return memo[amount]

    res = dp(amount)
    return res if res != float('inf') else -1 

# test
# 3
print(coinChange([1,2,5], 11))
# -1
print(coinChange([2], 3))
# 0
print(coinChange([1], 0))     

3
-1
0


### DP Tables

We can use DP table to avoid duplicated calculations. The idea is same as the memo.


In [23]:
def coinChange(coins, amount):
    dp = {i: float('inf') for i in range(1, amount+1)}
    dp[0] = 0

    for i in range(1, amount+1):
        for coin in coins:
            if i - coin < 0:
                continue
            dp[i] = min(dp[i], dp[i-coin] + 1)
    
    return dp[amount] if dp[amount] != float('inf') else -1

# test
# 3
print(coinChange([1,2,5], 11))
# -1
print(coinChange([2], 3))
# 0
print(coinChange([1], 0))     

3
-1
0
